In [114]:
import pandas as pd
from geopy.distance import geodesic
from sklearn.preprocessing import MinMaxScaler

In [116]:
events_df = pd.read_csv("events_permit_modified_final.csv")
orgs_df = pd.read_csv("DCLA_Cultural_Organizations_20250525.csv")


In [132]:
events_df = events_df.rename(columns=lambda x: x.rstrip())
orgs_df = orgs_df.rename(columns=lambda x: x.rstrip())

In [134]:
orgs_df.head()

,Organization Name,Address,City,State,Postcode,Main Phone #,Discipline,Council District,Community Board,Borough,Latitude,Longitude,Census Tract,BIN,BBL,NTA
0,24 Hour Company,151 Bank Street,New York,NY,10014,(646) 909-1321,Performing,Manhattan Council District #3,Manhattan Community Board #2,Manhattan,40.736112,-74.009053,79.0,1012047.0,1.006390e+09,West Village
1,"42nd Street Workshop, Inc.",421 Eighth Avenue,New York,NY,10001,(212) 695-4173,Performing,Manhattan Council District #3,Manhattan Community Board #4,Manhattan,40.751024,-73.994326,103.0,1084820.0,1.007550e+09,Hudson Yards-Chelsea-Flatiron-Union Square
2,"52nd Street Project, Inc.",789 Tenth Avenue,New York,NY,10019,(212) 333-5252,Performing,Manhattan Council District #3,Manhattan Community Board #4,Manhattan,40.766661,-73.990365,135.0,1087777.0,1.010818e+09,Clinton
3,"7 Loaves, Inc.","239 East 5th Street, #1D",New York,NY,10003,(212) 777-3891,Performing,Manhattan Council District #1,Manhattan Community Board #3,Manhattan,40.727323,-73.990251,38.0,1006629.0,1.004610e+09,East Village
4,A Gathering of the Tribes,285 East 3rd Street,New York,NY,10009,(212) 674-8262,Visual Arts,Manhattan Council District #2,Manhattan Community Board #3,Manhattan,40.721440,-73.980017,2601.0,1004405.0,1.003730e+09,Lower East Side


In [136]:
events_df.head()

,Event Type,Event Location,Latitude,Longitude,Cultural activity suggested,Date,Time,Date Timestamp,Nearest Taxi Zone,Taxi zone crowd score (out of 10),Creative score ( out of 10)
0,Marketplaces,MILL LANE between SOUTH WILLIAM STREET and STO...,40.704794,-74.009755,Art Sale,4/1/2023,11:00:00 AM,1/4/2023 11:00,NaN,NaN,NaN
1,Marketplaces,MILL LANE between SOUTH WILLIAM STREET and STO...,40.704794,-74.009755,Art Sale,4/1/2023,12:00:00 PM,1/4/2023 12:00,NaN,NaN,NaN
2,Marketplaces,MILL LANE between SOUTH WILLIAM STREET and STO...,40.704794,-74.009755,Art Sale,4/1/2023,1:00:00 PM,1/4/2023 13:00,NaN,NaN,NaN
3,Marketplaces,MILL LANE between SOUTH WILLIAM STREET and STO...,40.704794,-74.009755,Art Sale,4/1/2023,2:00:00 PM,1/4/2023 14:00,NaN,NaN,NaN
4,Marketplaces,MILL LANE between SOUTH WILLIAM STREET and STO...,40.704794,-74.009755,Art Sale,4/1/2023,3:00:00 PM,1/4/2023 15:00,NaN,NaN,NaN


In [138]:
events_df = events_df.rename(columns=lambda x: x.rstrip())

In [140]:
activity_discipline_map = {
    "Landscape painting": "Visual Arts",
    "Portrait painting": "Visual Arts",
    "Filmmaking": "Film/Video/Audio",
    "Portrait photography": "Photography",
    "street photography": "Photography",
    "Busking": "Performing",
    "Art Sale": "Visual Arts"
}

In [144]:
events_df["Discipline"] = events_df["Cultural activity suggested"].map(activity_discipline_map)
freq_map = (
    events_df.groupby(["Latitude", "Longitude", "Time"])
    .size()
    .reset_index(name="time_frequency")
)


In [148]:
events_df = events_df.merge(freq_map, on=["Latitude", "Longitude", "Time"], how="left")


In [152]:
def count_nearby_orgs(lat, lon, discipline, orgs, radius_m=1000):
    filtered_orgs = orgs[orgs["Discipline"] == discipline]
    if filtered_orgs.empty:
        return 0
    distances = filtered_orgs.apply(
        lambda row: geodesic((lat, lon), (row["Latitude"], row["Longitude"])).meters,
        axis=1
    )
    return (distances <= radius_m).sum()

events_df["nearby_org_count"] = events_df.apply(
    lambda row: count_nearby_orgs(row["Latitude"], row["Longitude"], row["Discipline"], orgs_df),
    axis=1
)


In [154]:
scaler = MinMaxScaler()
events_df[["norm_freq", "norm_orgs"]] = scaler.fit_transform(
    events_df[["time_frequency", "nearby_org_count"]]
)


In [156]:
events_df["raw_score"] = 0.7 * events_df["norm_freq"] + 0.3 * events_df["norm_orgs"]


In [158]:
final_scaler = MinMaxScaler(feature_range=(2, 10))
events_df["cultural_activity_score"] = final_scaler.fit_transform(events_df[["raw_score"]])

In [110]:
result = events_df[[
    "timestamp", "activity", "latitude", "longitude", "discipline",
    "time_frequency", "nearby_org_count", "cultural_activity_score"
]]

KeyError: "['timestamp', 'activity', 'latitude', 'longitude', 'discipline'] not in index"

In [160]:
events_df.to_csv("score.csv")